# 言語理解（機械学習による方法：ドメイン推定）
ここでは、言語理解を機械学習による方法で実装します。サンプルのデータセットとしてレストラン検索と天気案内のユーザ発話およびそのアノテーションデータを用意しました。このデータでは既にMeCab（標準搭載の辞書）で単語分割が行われています。まずは、これら２つを分類するドメイン推定を実装します。特徴量には、Bag-of-words、学習済みWord2vecを用います。

## 事前の設定
- 必要なpythonライブラリのインストール
    - scikit-learn
    - numpy
    - gensim
- MeCabのインストール
- MeCabをpythonで使用するためのmecab-pythonのインストール

## 機械学習の手順

- 処理の手順は以下の通りです。
    - データを読み込む
    - 学習とテストデータに分割（80対20）
    - 入力データを特徴量に変換する
    - 出力データをラベルに変換する
    - 機械学習ライブラリを用いてモデルを学習する
    - 学習したモデルをテストデータで評価する

In [1]:
# 必要なラブラリを読み込む

import sys, os
import re
import numpy as np
import pickle

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from gensim.models import KeyedVectors

import MeCab


In [2]:
# 変数の設定

NUM_TRAIN = 80      # 100個のデータのうち最初の80個を学習に利用
NUM_TEST = 20       # 100個のデータのうち残りの20個をテストに利用

LABEL_RESTAURANT = 0    # レストラン検索ドメインのラベル
LABEL_WEATHER = 1       # 天気案内ドメインのラベル

In [3]:
# データを読み込む

# レストランデータ
with open('./data/slu-restaurant-annotated.csv', 'r', encoding='utf-8') as f:
    lines_restaurant = f.readlines()

with open('./data/slu-weather-annotated.csv', 'r', encoding='utf-8') as f:
    lines_weather = f.readlines()

# 学習とテストデータに分割（80対20）
# 注）本来は交差検定を行うことが望ましい
lines_restaurant_train = lines_restaurant[:NUM_TRAIN]
lines_restaurant_test = lines_restaurant[NUM_TRAIN:]
lines_weather_train = lines_weather[:NUM_TRAIN]
lines_weather_test = lines_weather[NUM_TRAIN:]

data_train = []
for line in lines_restaurant_train:

    # 既に分割済みの単語系列を使用
    d = line.strip().split(',')[2].split('/')
    
    # 入力単語系列と正解ラベルのペアを格納
    data_train.append([d, LABEL_RESTAURANT])

# 以下同様
for line in lines_weather_train:
    d = line.strip().split(',')[2].split('/')
    data_train.append([d, LABEL_WEATHER])

data_test = []
for line in lines_restaurant_test:
    d = line.strip().split(',')[2].split('/')
    data_test.append([d, LABEL_RESTAURANT])

for line in lines_weather_test:
    d = line.strip().split(',')[2].split('/')
    data_test.append([d, LABEL_WEATHER])

# 最初のデータだけ表示
print(data_train[0])
print(data_test[0])


[['京大', '近く', 'の', 'うどん', '屋', 'を', '教え', 'て'], 0]
[['から', '揚げ', 'が', 'おいしい', 'お', '店', 'を', '教え', 'て', 'ください'], 0]


In [4]:
# Bag-of-Words表現を作成する

# 学習データの単語を語彙（カバーする単語）とする
word_list = {}

for data in data_train:
    for word in data[0]:
        word_list[word] = 1

print(word_list.keys())

# 単語とそのインデクスを作成する
word_index = {}
for idx, word in enumerate(word_list.keys()):
    word_index[word] = idx

print(word_index)

# ベクトルの次元数（未知語を扱うためにプラス１）
vec_len = len(word_list.keys()) + 1
print(vec_len)

dict_keys(['京大', '近く', 'の', 'うどん', '屋', 'を', '教え', 'て', '四条', '付近', 'に', '美味しい', '餃子', 'さん', 'は', 'あり', 'ます', 'か', '京都', '駅', 'で', '牛', '丼', '三条', 'カレー', '探し', 'い', '金閣寺', 'ラーメン', 'ハンバーガ', 'お', '寿司', '銀閣寺', '高級', '料亭', '場所', '検索', 'し', '御所', '周り', '定食', '行き', 'たい', '清水寺', '周辺', 'ステーキ', 'な', '蕎麦', '祇園', 'ハンバーグ', '二条城', '百', '万', '遍', 'どこ', '出町柳', '食堂', 'へ', '大学', 'カフェ', '喫茶店', 'とか', 'ない', 'イタリアン', 'レストラン', '烏丸', 'あたり', 'フランス', '料理', 'ん', 'だ', 'けど', '天丼', '食べ', '一乗寺', '個室', 'ある', '中華', 'あっさり', 'た', 'もの', '評判', 'こってり', '系', 'スペイン', '通', 'ガッツリ', '店', '穴子', 'が', 'られる', 'って', 'この', '格安', '人気', '安く', '安い', '居酒屋', '手頃', '価格', 'すき焼き', 'ケーキ', 'ところ', '回転', 'ここら', 'へん', '5000', '円', '以内', '掘り', 'ご', 'たつ', '形式', '湯豆腐', '川床', '鴨川', '沿い', '郷土', '出し', 'くれる', '二', '条', '通り', 'アフタヌーン', 'ティー', '飲める', '1', 'フレンチ', '野外', '焼肉', 'さっぱり', 'スイーツ', '広い', '庭', 'バーベキュー', '市内', '魚', '創作', 'お願い', '屋上', 'やっ', 'いる', 'ビアガーデン', '東山', 'おすすめ', '東福寺', 'さくっと', 'ファースト', 'フード', '河原町', 'ネパール', 'たべ', 'スイス', '北山', 'ばん', 'ざい', 'い

In [5]:
# 単語の系列とBag-of-Words表現を作成するための情報を受け取りベクトルを返す関数を定義
def make_bag_of_words(words, vocab, dim, pos_unk):

    vec = [0] * dim
    for w in words:

        # 未知語
        if w not in vocab:
            vec[pos_unk] = 1
        
        # 学習データに含まれる単語
        else:
            vec[vocab[w]] = 1
    
    return vec

# 試しに変換してみる
feature_vec = make_bag_of_words(data_train[0][0], word_index, vec_len, vec_len)
print(feature_vec)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
# 学習データをBoW表現に変換する
data_train_bow = []
for data in data_train:
    feature_vec = make_bag_of_words(data[0], word_index, vec_len, vec_len-1)
    data_train_bow.append([feature_vec, data[1]])

In [7]:
# 入力と正解ラベルで別々のデータにする
train_x = [d[0] for d in data_train_bow]
train_y = [d[1] for d in data_train_bow]

# 学習サンプル数
print(len(train_x))

160


In [8]:
# SVMによる学習
# 注）実際にはパラメータの調整が必要だが今回は行わない
clf = svm.SVC()
clf.fit(train_x, train_y) 

# 学習済みモデルを保存
filename = './data/slu-domain-svm.model'
pickle.dump(clf, open(filename, 'wb'))

In [9]:
# テストデータの作成

data_test_bow = []
for data in data_test:
    feature_vec = make_bag_of_words(data[0], word_index, vec_len, vec_len-1)
    data_test_bow.append([feature_vec, data[1]])

test_x = [d[0] for d in data_test_bow]
test_y = [d[1] for d in data_test_bow]

# テストサンプル数
print(len(test_x))

40


In [10]:
# テストデータでの評価
predict_y = clf.predict(test_x)

# 評価結果を表示
target_names = ['restaurant', 'weather']
print(classification_report(test_y, predict_y, target_names=target_names))

              precision    recall  f1-score   support

  restaurant       0.90      0.95      0.93        20
     weather       0.95      0.90      0.92        20

    accuracy                           0.93        40
   macro avg       0.93      0.93      0.92        40
weighted avg       0.93      0.93      0.92        40



### ロジスティック回帰の利用
SVMではなくロジスティック回帰で同様の学習とテストを実行してみます。分布の形状がモデル化さている分、少ないデータ数でも頑健に学習することが期待されます。

In [11]:
# LogisticRegressionによる学習と評価
# 注）実際にはパラメータの調整が必要だが今回は行わない
clf_lr = LogisticRegression()
clf_lr.fit(train_x, train_y) 

# 学習済みモデルを保存
filename = './data/slu-domain-lr.model'
pickle.dump(clf, open(filename, 'wb'))

# テストデータでの評価
predict_y = clf_lr.predict(test_x)

# 評価結果を表示
target_names = ['restaurant', 'weather']
print(classification_report(test_y, predict_y, target_names=target_names))

              precision    recall  f1-score   support

  restaurant       0.90      0.95      0.93        20
     weather       0.95      0.90      0.92        20

    accuracy                           0.93        40
   macro avg       0.93      0.93      0.92        40
weighted avg       0.93      0.93      0.92        40



### Word2vecの利用
次に，特徴量としてBag-of-wordsではなくWord2vecを用いてみます。日本語でも様々な学習済みモデルがありますが、ここでは下記のものを用います。Word2vecは単語間の距離（類似性）を考慮することができるため、Bag-of-wordsよりもより頑健になることが期待されます。

- 日本語 Wikipedia エンティティベクトル
    - http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/
    - 最新のモデルをダウンロードして解凍してください

In [12]:
# 学習済みWord2vecファイルを読み込む
model_filename = './data/entity_vector.model.bin'
model_w2v = KeyedVectors.load_word2vec_format(model_filename, binary=True)

# 単語ベクトルの次元数
print(model_w2v.vector_size)

200


In [13]:
# Word2vecで特徴量を作成する関数を定義
# ここでは文内の各単語のWord2vecを足し合わせたものを文ベクトルととして利用する
def make_sentence_vec_with_w2v(words, model_w2v):

    sentence_vec = np.zeros(model_w2v.vector_size)
    num_valid_word = 0
    for w in words:
        if w in model_w2v:
            sentence_vec += model_w2v[w]
            num_valid_word += 1
    
    # 有効な単語数で割
    sentence_vec /= num_valid_word
    return sentence_vec


# 試しに変換してみる
feature_vec = make_sentence_vec_with_w2v(data_train[0][0], model_w2v)
print(feature_vec)

[-0.77434402 -0.15547702  0.38850617 -1.45265855  0.50567805 -1.07454265
  0.70662174  1.82708089 -0.86861591 -0.57705    -0.35228045  0.38542001
 -1.47540402 -0.17376012  1.24027113  0.5010112   0.92485247 -0.94757835
  0.02257733  0.58819923  0.08229128  0.9749545   1.0750578   1.06965833
  0.99780852  1.10840474  0.2614957   0.36480585 -1.79708698  0.51195106
 -0.5499662   0.06639886 -1.29126327  1.45213751  0.95796485 -0.95632624
 -0.74981602  0.04241584  1.19777854 -0.5407796   2.45429978  0.22389831
 -0.99984368 -1.21721396  0.33467129  1.13219028  0.12149247  0.23289285
 -1.11721854 -0.48681203  0.35655155 -1.34670382  1.60762816 -0.29011638
  0.36180672 -1.86755647  0.08381788  0.29022732 -0.91748604  0.74968182
 -1.42012271 -0.15379699  1.10182209 -0.08516333 -1.05848608 -0.92968358
 -1.66354649  0.52094843  0.28299145  1.59462082 -1.40888757 -1.28874561
 -1.05283381  0.23811523  0.94129454 -0.13580772  1.09463759 -0.22150378
  1.09775052 -0.82774358  0.21583352  0.49302975 -0

In [14]:
# Word2vecを用いて学習を行う

data_train_w2v = []
for data in data_train:
    feature_vec = make_sentence_vec_with_w2v(data[0], model_w2v)
    data_train_w2v.append([feature_vec, data[1]])

train_x = [d[0] for d in data_train_w2v]
train_y = [d[1] for d in data_train_w2v]

clf = svm.SVC()
clf.fit(train_x, train_y) 

filename = './data/slu-domain-svm-word2vec.model'
pickle.dump(clf, open(filename, 'wb'))


In [15]:
# テストデータでの評価

data_test_w2v = []
for data in data_test:
    feature_vec = make_sentence_vec_with_w2v(data[0], model_w2v)
    data_test_w2v.append([feature_vec, data[1]])

test_x = [d[0] for d in data_test_w2v]
test_y = [d[1] for d in data_test_w2v]

# テストデータでの評価
predict_y = clf.predict(test_x)

# 評価結果を表示
target_names = ['restaurant', 'weather']
print(classification_report(test_y, predict_y, target_names=target_names))

              precision    recall  f1-score   support

  restaurant       1.00      1.00      1.00        20
     weather       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40



In [16]:
# 自由なデータで試してみる

# 入力データ
test_input_list = [
    '京都駅周辺で美味しいラーメン屋さんを教えて',
    '横浜は晴れていますか'
]

# MeCabによる分割と特徴量抽出
m = MeCab.Tagger ("-Owakati")
test_x = []
for d in test_input_list:
    words_input = m.parse(d).strip().split(' ')
    feature_vec = make_sentence_vec_with_w2v(words_input, model_w2v)
    test_x.append(feature_vec)

# 予測
predict_y = clf.predict(test_x)

for result, text in zip(predict_y, test_input_list):
    
    print('Input: ' + text)
    
    if result == LABEL_RESTAURANT:
        print('Estimated domain: Restaurant')
    elif result == LABEL_WEATHER:
        print('Estimated domain: Weather')
    
    print()

Input: 京都駅周辺で美味しいラーメン屋さんを教えて
Estimated domain: Restaurant

Input: 横浜は晴れていますか
Estimated domain: Weather

